In [ ]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

In [ ]:
monkey.patch_all()
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = True
server.serve_forever()
#http://127.0.0.2:5000/

The history saving thread hit an unexpected error (LoopExit('This operation would block forever', <Hub at 0x19e61b0e2a8 select pending=0 ref=0>)).History will not be written to the database.


127.0.0.1 - - [2020-02-12 14:10:38] "GET / HTTP/1.1" 200 6816 0.016015
127.0.0.1 - - [2020-02-12 14:10:38] "GET /static/favicon-16x16.png HTTP/1.1" 200 664 0.002004
127.0.0.1 - - [2020-02-12 14:10:50] "GET /bonds HTTP/1.1" 200 6171 0.001992


In [ ]:
import sys
sys.path.insert(1, r'C:\Users\Kart\jupyter_projects')
import mygit as mg

In [ ]:
folder=r'C:\Users\Kart\jupyter_projects\webtrade'
git_url='git@github.com:Rayveni/web.git'
gt=mg.gitwrapper(folder)
gt.push_commit(git_url,commit_msg='update')

In [ ]:
from external_sources import smartlab

tv=smartlab()
a=tv.bonds_info()

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\src")

In [ ]:
from commons import convert_file_size,init_db_manager
from jobs import *
a=job_sectors(init_db_manager())